In [1]:
# importamos las librerías necesarias
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics.pairwise import cosine_similarity
from operator import itemgetter
import matplotlib.pyplot as plt 
import seaborn as sns
np.random.seed(639)
from sklearn.preprocessing import StandardScaler
from scipy import stats
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')
import spacy
import re
from unidecode import unidecode

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alter\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# creamos dataframe con los nombres de los juejos
df_names = pd.read_parquet('item_names.parquet')
df_names.head()

,item_id,item_name
0,761140,Lost Summoner Kitty
1,643980,Ironbound
2,670290,Real Pool 3D - Poolians
3,767400,弹炸人2222
4,772540,Battle Royale Trainer


In [36]:
len(df_names)

31220

In [3]:
# renombramos columnas del dataframe
df_names.rename(columns= {'item_id' : 'id', 'item_name': 'app_name'}, inplace = True)

In [4]:
# creamos dataframe con las features de los juegos
df = pd.read_parquet('../CleanData/steam_games.parquet')
df.head()

,id,app_name,title,price,developer,release_date,tag_1980s,tag_1990's,tag_2.5D,tag_2D,...,spec_Single-player,spec_Stats,spec_Steam Achievements,spec_Steam Cloud,spec_Steam Leaderboards,spec_Steam Trading Cards,spec_Steam Turn Notifications,spec_Steam Workshop,spec_SteamVR Collectibles,spec_Valve Anti-Cheat enabled
0,761140,Lost Summoner Kitty,Lost Summoner Kitty,4.99,Kotoshiro,2018-01-04,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,643980,Ironbound,Ironbound,0.00,Secret Level SRL,2018-01-04,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,670290,Real Pool 3D - Poolians,Real Pool 3D - Poolians,0.00,Poolians.com,2017-07-24,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,767400,弹炸人2222,弹炸人2222,0.99,彼岸领域,2017-12-07,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,772540,Battle Royale Trainer,Battle Royale Trainer,3.99,Trickjump Games Ltd,2018-01-04,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
df_names = df[['id', 'app_name']]
df_names.head()

,id,app_name
0,761140,Lost Summoner Kitty
1,643980,Ironbound
2,670290,Real Pool 3D - Poolians
3,767400,弹炸人2222
4,772540,Battle Royale Trainer


In [44]:
df_names.to_parquet('item_names.parquet')

In [5]:
# eliminamos columnas y lo combinamos con el dataframe de nombres
df.drop(columns=['app_name', 'title'], inplace=True)
df = df_names.merge(df, how='left')
df.head()

,id,app_name,price,developer,release_date,tag_1980s,tag_1990's,tag_2.5D,tag_2D,tag_2D Fighter,...,spec_Single-player,spec_Stats,spec_Steam Achievements,spec_Steam Cloud,spec_Steam Leaderboards,spec_Steam Trading Cards,spec_Steam Turn Notifications,spec_Steam Workshop,spec_SteamVR Collectibles,spec_Valve Anti-Cheat enabled
0,761140,Lost Summoner Kitty,4.99,Kotoshiro,2018-01-04,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,643980,Ironbound,0.00,Secret Level SRL,2018-01-04,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,670290,Real Pool 3D - Poolians,0.00,Poolians.com,2017-07-24,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,767400,弹炸人2222,0.99,彼岸领域,2017-12-07,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,772540,Battle Royale Trainer,3.99,Trickjump Games Ltd,2018-01-04,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# creamos la columna año y eliminamos la columna release_date
df['Year'] = df['release_date'].dt.year
df.drop(columns='release_date', inplace= True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31220 entries, 0 to 31219
Columns: 495 entries, id to Year
dtypes: float64(492), int32(1), object(2)
memory usage: 117.8+ MB


In [8]:
# importamos en un dataframe otro archivo con features de item
df_features = pd.read_parquet('item_item_features.parquet')
df_features.rename(columns={'item_id': 'id'}, inplace= True)
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10978 entries, 0 to 10977
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   id                10978 non-null  int32
 1   playtime_forever  10978 non-null  int64
 2   playtime_2weeks   10978 non-null  int64
dtypes: int32(1), int64(2)
memory usage: 214.5 KB


In [9]:
df_features.head()

,id,playtime_forever,playtime_2weeks
0,10,17107858,46656
1,20,960524,14117
2,30,756375,14042
3,40,154424,38
4,50,726545,2485


In [10]:
# lo combinamos con el dataframe original
df = df.merge(df_features, how='left')
df.head()

,id,app_name,price,developer,tag_1980s,tag_1990's,tag_2.5D,tag_2D,tag_2D Fighter,tag_3D Platformer,...,spec_Steam Cloud,spec_Steam Leaderboards,spec_Steam Trading Cards,spec_Steam Turn Notifications,spec_Steam Workshop,spec_SteamVR Collectibles,spec_Valve Anti-Cheat enabled,Year,playtime_forever,playtime_2weeks
0,761140,Lost Summoner Kitty,4.99,Kotoshiro,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018.0,NaN,NaN
1,643980,Ironbound,0.00,Secret Level SRL,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018.0,NaN,NaN
2,670290,Real Pool 3D - Poolians,0.00,Poolians.com,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017.0,NaN,NaN
3,767400,弹炸人2222,0.99,彼岸领域,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017.0,NaN,NaN
4,772540,Battle Royale Trainer,3.99,Trickjump Games Ltd,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018.0,NaN,NaN


In [11]:
df.duplicated().sum()               # vemos que no hay duplicados

0

In [17]:
# importamos los datos de análisis de sentimiento
df_sent = pd.read_parquet('../sentiment_analysis_2.parquet')
df_sent.head()

,user_id,item_id,sentiment_analysis_2,recommend,Año
0,76561197970982479,1250,2,1,2011
1,76561197970982479,22200,2,1,2011
2,76561197970982479,43110,2,1,2011
3,js41637,251610,2,1,2014
4,js41637,227300,2,1,2013


In [13]:
# generamos las features rating y positive_ratio
df_sent.drop(columns=['user_id', 'Año'], inplace=True)
df_sent['positive_ratio'] = np.where(df_sent['sentiment_analysis_2'] == 2, 1, 0)
df_sent['cant_reviews'] = np.where(df_sent['item_id'] != 0, 1, 0)
df_sent_grouped = df_sent.groupby('item_id').agg(sum)
df_sent_grouped.rename(columns={'sentiment_analysis_2': 'rating'}, inplace=True)
df_sent_grouped.reset_index(inplace=True)
df_sent_grouped.rename(columns={'item_id': 'id'}, inplace=True)

In [18]:
df_sent_grouped['positive_ratio'] = df_sent_grouped['positive_ratio'] / df_sent_grouped['cant_reviews']
df_sent_grouped.drop(columns='cant_reviews', inplace= True)

In [19]:
df_sent_grouped.head()

,id,rating,recommend,positive_ratio
0,10,89,55,0.660714
1,20,26,11,0.647059
2,30,7,3,0.750000
3,40,2,1,1.000000
4,50,8,3,1.000000


In [20]:
# combinamos con el dataframe de features que estamos construyendo
df = df.merge(df_sent_grouped, how='left')
df.head()

,id,app_name,price,developer,tag_1980s,tag_1990's,tag_2.5D,tag_2D,tag_2D Fighter,tag_3D Platformer,...,spec_Steam Turn Notifications,spec_Steam Workshop,spec_SteamVR Collectibles,spec_Valve Anti-Cheat enabled,Year,playtime_forever,playtime_2weeks,rating,recommend,positive_ratio
0,761140,Lost Summoner Kitty,4.99,Kotoshiro,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2018.0,NaN,NaN,NaN,NaN,NaN
1,643980,Ironbound,0.00,Secret Level SRL,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2018.0,NaN,NaN,NaN,NaN,NaN
2,670290,Real Pool 3D - Poolians,0.00,Poolians.com,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2017.0,NaN,NaN,NaN,NaN,NaN
3,767400,弹炸人2222,0.99,彼岸领域,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2017.0,NaN,NaN,NaN,NaN,NaN
4,772540,Battle Royale Trainer,3.99,Trickjump Games Ltd,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2018.0,NaN,NaN,NaN,NaN,NaN


In [22]:
# En esta sección se pueden generar features con las palabras de los nombres 
stopwords = nltk.corpus.stopwords.words('english')

def clean_text(name):
    if pd.isna(name) or name is None:
        return name
    else:
        # limpieza
        name = str(name)
        name = name.lower()
        name= name.replace('\n', '')
        name = unidecode(name)
        name = re.sub(r'https?://\S+|www\.\S+', '', name)

        # tokenización
        name = nltk.tokenize.RegexpTokenizer("[\w]+").tokenize(name)

        # quitar stopwords
        name = [word.strip() for word in name if word not in stopwords]
        return name

nlp = spacy.load('en_core_web_sm')          # cargamos nlp en inglés

# definimos una función para cambiar la función posicional de las palabras a una expresión de una sola letra
def get_wordnet_pos(pos):
    if pos == 'VERB':
        return 'v'
    elif pos == 'ADJ':
        return 'a'
    elif pos == 'ADV':
        return 'r'
    elif pos == 'NOUN':
        return 'n'
    else:
        return 'n'

# definimos una función para lemmatizar el texto de cada review
def lemmatize_text(name):
    if name is None:
        return name

    lemmatizer = WordNetLemmatizer()

    # Obtener posiciones gramaticales en un conjunto
    pos_set = {'VERB', 'ADJ', 'ADV', 'NOUN'}

    # Lematizar en lotes usando spaCy
    doc = nlp(' '.join(name))

    lemmatized_words = [lemmatizer.lemmatize(token.text, get_wordnet_pos(token.pos_)) for token in doc if token.pos_ in pos_set]

    return lemmatized_words


In [25]:
df['name_feature'] = df['app_name'].apply(clean_text)

In [26]:
df['name_feature'] = df['name_feature'].apply(lemmatize_text)

In [28]:
df['name_feature'] = df['name_feature'].apply(lambda x: ' '.join(x) if x else '')
dummy_df = pd.get_dummies(df['name_feature'].str.split(expand=True).stack(), prefix='game')
dummy_df.head()

game_08s  game_0one  game_10c  game_10th  game_111th  game_11th  \
0 0     False      False     False      False       False      False   
  1     False      False     False      False       False      False   
  2     False      False     False      False       False      False   
1 0     False      False     False      False       False      False   
2 0     False      False     False      False       False      False   

     game_12ga  game_13th  game_15th  game_16bit  ...  game_zroll  \
0 0      False      False      False       False  ...       False   
  1      False      False      False       False  ...       False   
  2      False      False      False       False  ...       False   
1 0      False      False      False       False  ...       False   
2 0      False      False      False       False  ...       False   

     game_zubmariner  game_zulu  game_zumbi  game_zup  \
0 0            False      False       False     False   
  1            False      False       False     False   
  2            False      False       False     False   
1 0            False      False       False     False   
2 0            False      False       False     False   

     game_zusupeshiyarubgmpatsuku  game_zxill  game_zyd  game_zylon  game_zzzz  
0 0                         False       False     False       False      False  
  1                         False       False     False       False      False  
  2                         False       False     False       False      False  
1 0                         False       False     False       False      False  
2 0                         False       False     False       False      False  

[5 rows x 11989 columns]

In [ ]:
# no generaremos las features de palabras ya que son mucha cantidad de columnas y computacionalmente será muy costoso para nuestra utilidad

In [21]:
# eliminamos las columnas 'id' y 'app_name' ya que no son features
df_items = df.drop(columns='id')
df_items = df_items.drop(columns='app_name')

In [22]:
df_items['playtime_forever'].fillna(0, inplace=True)                                # imputamos nulos con cero
df_items['playtime_2weeks'].fillna(0, inplace=True)                                 # imputamos nulos con cero
df_items['recommend'].fillna(np.nanmedian(df_items['recommend']), inplace=True)     # imputamos nulos con la mediana
df_items['rating'].fillna(np.nanmedian(df_items['rating']), inplace=True)           # imputamos nulos con la mediana

In [23]:
scaler = StandardScaler()       # escalamos con z-score
df_items['playtime_forever'] = scaler.fit_transform(df_items['playtime_forever'].array.reshape(-1,1))
scaler = StandardScaler()       # escalamos con z-score
df_items['playtime_2weeks'] = scaler.fit_transform(df_items['playtime_2weeks'].array.reshape(-1,1))

In [24]:
scaler = StandardScaler()       # escalamos con z-score
df_items['recommend'] = scaler.fit_transform(df_items['recommend'].array.reshape(-1,1))
scaler = StandardScaler()       # escalamos con z-score
df_items['rating'] = scaler.fit_transform(df_items['rating'].array.reshape(-1,1))

In [25]:
df_items.drop(columns='developer', inplace=True)        # eliminamos la columna developer que tampoco es feature

In [26]:
# vemos como queda nuestro dataframe con features de items
df_items.head()

,price,tag_1980s,tag_1990's,tag_2.5D,tag_2D,tag_2D Fighter,tag_3D Platformer,tag_3D Vision,tag_4 Player Local,tag_4X,...,spec_Steam Turn Notifications,spec_Steam Workshop,spec_SteamVR Collectibles,spec_Valve Anti-Cheat enabled,Year,playtime_forever,playtime_2weeks,rating,recommend,positive_ratio
0,4.99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2018.0,-0.02928,-0.028227,-0.043406,-0.041109,NaN
1,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2018.0,-0.02928,-0.028227,-0.043406,-0.041109,NaN
2,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2017.0,-0.02928,-0.028227,-0.043406,-0.041109,NaN
3,0.99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2017.0,-0.02928,-0.028227,-0.043406,-0.041109,NaN
4,3.99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2018.0,-0.02928,-0.028227,-0.043406,-0.041109,NaN


In [27]:
df_items.isna().sum()

price                2321
tag_1980s            2321
tag_1990's           2321
tag_2.5D             2321
tag_2D               2321
                    ...  
playtime_forever        0
playtime_2weeks         0
rating                  0
recommend               0
positive_ratio      27636
Length: 497, dtype: int64

In [28]:
df_items['price'].fillna(np.nanmedian(df_items['price']), inplace=True)     # completamos nulos de precio con mediana

In [29]:
df_items['Year'].fillna(stats.mode(df_items['Year'])[0], inplace=True)      # completamos nulos de año con la moda

In [30]:
df_items.columns[df_items.isnull().any()].tolist()                          # obtenemos la lista de columnas con nulos

['tag_1980s',
 "tag_1990's",
 'tag_2.5D',
 'tag_2D',
 'tag_2D Fighter',
 'tag_3D Platformer',
 'tag_3D Vision',
 'tag_4 Player Local',
 'tag_4X',
 'tag_6DOF',
 'tag_Abstract',
 'tag_Accounting',
 'tag_Action',
 'tag_Action RPG',
 'tag_Action-Adventure',
 'tag_Adventure',
 'tag_Agriculture',
 'tag_Aliens',
 'tag_Alternate History',
 'tag_America',
 'tag_Animation & Modeling',
 'tag_Anime',
 'tag_Arcade',
 'tag_Arena Shooter',
 'tag_Artificial Intelligence',
 'tag_Assassin',
 'tag_Asynchronous Multiplayer',
 'tag_Atmospheric',
 'tag_Audio Production',
 'tag_Base Building',
 'tag_Based On A Novel',
 'tag_Basketball',
 'tag_Batman',
 "tag_Beat 'em up",
 'tag_Benchmark',
 'tag_Blood',
 'tag_Board Game',
 'tag_Bowling',
 'tag_Building',
 'tag_Bullet Hell',
 'tag_Bullet Time',
 'tag_CRPG',
 'tag_Capitalism',
 'tag_Card Game',
 'tag_Cartoon',
 'tag_Cartoony',
 'tag_Casual',
 'tag_Character Action Game',
 'tag_Character Customization',
 'tag_Chess',
 'tag_Choices Matter',
 'tag_Choose Your Own 

In [31]:
df_items.fillna(0, inplace=True)                    # completamos las features que faltan con 0

In [32]:
len(df_items)

31220

In [33]:
df_items.to_parquet('item_features_complete.parquet')

In [43]:
pd.set_option('display.max_columns', None)

In [34]:
#definimos una función para obtener items similares según cosine similarity
def item_item_recom(item_id, n_recom=5, umbral= 0.9999999):
    
    try: 
        item_id = int(item_id)
        indice = df_names[df_names['id'] == item_id].index[0]       # verificamos que el item_id exista
    except Exception as e:
        print(f'Error: {e}')
        return print('Ingrese un Id de producto válido')            # retornamos mensaje en caso de no existir el id
    
    similaridades = {}                                              # iniciamos diccionario de similaridades
    contador = 0                                                    # contador para realizar corte por umbral
    for i in range(len(df_items)):                      # recorremos el dataframe
        if i != indice:                                 # no tomamos el indice del juego de entrada
            sim = cosine_similarity(df_items.iloc[indice,:-4].values.reshape(1,-1), df_items.iloc[i,:-4].values.reshape(1,-1))[0][0]
            similaridades[i] = sim                      # calclualmos la similaridad y la guardamos en el diccionario
            if sim > umbral:                            # verificamos si la similaridad es mayor al umbral predeterimnado
                contador += 1                           # si es mayor, sumamos 1 al contador
            if contador > n_recom:                      # si se supera la cantidad de recomendaciones del contador, detenemos la búsqueda
                break
    
    similaridades_sorted = sorted(similaridades.items(), key= itemgetter(1), reverse=True)      # ordenamos las similaridades por valores

    items_recomendados = []                             # creamos diccionario vacío para acumular los juegos recomendados

    for i in range(n_recom):                            # nos quedamos con los indices de los n_recom primeros juegos
        items_recomendados.append(similaridades_sorted[i][0])
        
    return df_names.loc[items_recomendados,'app_name']  # retornamos dataframe de recomendados

In [42]:
item_item_recom(730)

47                       Garry's Mod
28048                       Terraria
28933    The Elder Scrolls V: Skyrim
27067                       Warframe
28789         Counter-Strike: Source
Name: app_name, dtype: object

In [39]:
df_names.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31220 entries, 0 to 5145725
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        31220 non-null  int32 
 1   app_name  31220 non-null  object
dtypes: int32(1), object(1)
memory usage: 1.6+ MB


In [76]:
df_names.iloc[[1033, 11036, 12608, 5410, 6388, 3253],:]

,id,app_name
1033,730,Counter-Strike: Global Offensive
11036,716660,"Warhammer 40,000: Sanctus Reach - Sons of Cadia"
12608,720680,LEGO® Marvel Super Heroes 2 - Season Pass
5410,434061,OMSI 2 Add-On Citybus O405/O405G
6388,458860,Order of Battle: Winter War
3253,339200,Oceanhorn: Monster of Uncharted Seas
